In [7]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

sys.path.append('../Scripts')
from Data_Processing import DataProcessing

from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
from datetime import datetime
from sklearn.preprocessing import PowerTransformer

import joblib

import warnings
warnings.filterwarnings('ignore')

In [8]:
ColumnTransformer = joblib.load('../Models/Column_Transformer.pkl')
#PowerTransformer = joblib.load('../Models/Power_Transformer.pkl')
ColumnTransformer_NN = joblib.load('../Models/Column_Transformer_NN.pkl')

In [9]:
df = DataProcessing('../Data/test.csv')

In [4]:
y = df['Lap_Time']
X = df.drop(columns=['Lap_Time'])

obj_columns = list(X.select_dtypes(include=object).columns)
obj_columns.append('Lap_Number')
obj_columns.append('Lap_Improvement')

num_columns = list(X.select_dtypes(include='number').columns)
num_columns.remove('Lap_Number')
num_columns.remove('Lap_Improvement')

In [10]:
#NN Only
y = df['Lap_Time']
X = df.drop(columns=['Lap_Time'])

obj_columns = list(X.select_dtypes(include=object).columns)

obj_columns.append('Lap_Improvement')
obj_columns.append('Lap_Number')
obj_columns.append('S1_Improvement')
obj_columns.append('S2_Improvement')
obj_columns.append('S3_Improvement')

num_columns = list(X.select_dtypes(include='number').columns)
num_columns.remove('Lap_Number')
num_columns.remove('Lap_Improvement')
num_columns.remove('S1_Improvement')
num_columns.remove('S2_Improvement')
num_columns.remove('S3_Improvement')

In [11]:
#X[num_columns] = PowerTransformer.transform(X[num_columns])
trans_X_nn = ColumnTransformer_NN.transform(X)

#trans_X = trans_X.toarray()
#trans_X = trans_X[:,[0, 2, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 72, 73]]
#trans_X_nn = trans_X_nn.toarray()

In [ ]:
def root_mean_squared_log_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

In [12]:
#Neural Network
nn_model = load_model('../Models/NN_model_test.h5')

#nn_model = load_model('../Models/NN_model.h5', custom_objects={'root_mean_squared_log_error': root_mean_squared_log_error})

In [ ]:
#Random Forest
rf_model = joblib.load('../Models/RF_Model.h5')

In [ ]:
#Gradient Boost
gb_model = joblib.load('../Models/Gradient_Boost_Model.h5')

In [13]:
nn_y_scaler = joblib.load('../Models/NN_Y_Scaler.pkl')

y_predicted_nn = nn_y_scaler.inverse_transform(nn_model.predict(trans_X_nn))
y_predicted_nn = ((1 / y_predicted_nn) - 1).ravel()
y_predicted_nn = nn_y_scaler.inverse_transform(nn_model.predict(trans_X_nn))
#y_predicted_rf = rf_model.predict(trans_X)
#y_predicted_gb = gb_model.predict(trans_X)

In [ ]:
results = pd.DataFrame()
results['NN'] = y_predicted_nn
results['RF'] = y_predicted_rf
results['GB'] = y_predicted_gb
results['LAP_TIME'] = (results['NN'] + results['RF'] + results['GB']) / 3
submission = results[['LAP_TIME']]
results

In [ ]:
#Random Forest Only
submission = results[['RF']]
submission = submission.rename(columns={'RF': 'LAP_TIME'})

In [ ]:
today = datetime.today().strftime('%m-%d-%y %H-%M')
submission.to_csv(f'../Submissions/Dare_In_Reality {today}.csv', index=False)

### Just Neural Network

In [15]:
submission = pd.DataFrame()
submission['LAP_TIME'] = y_predicted_nn.ravel()
submission

,LAP_TIME
0,100.422600
1,101.474510
2,103.890556
3,99.426315
4,101.097679
...,...
415,99.112122
416,96.492813
417,101.039185
418,97.143478


In [16]:
submission.to_csv(f'../Submissions/Dare_In_Reality NN Only.csv', index=False)

In [ ]:
y_predicted_nn

In [ ]:
df